In [52]:
import re
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

# 1. SETUP: Regex and Helpers
# CORRECTED: Logs start with "- -" (no IP), then timestamp, then method, path, status, size, referer, session_id, user_agent
LOG_PATTERN = r'- - \[(?P<timestamp>.*?)\] "(?P<method>\w+)\s+(?P<path>.*?)\s+HTTP\/.*?"\s+(?P<status>\d+)\s+(?P<size>\d+)\s+"(?P<referer>.*?)"\s+(?P<session_id>[\w-]+)\s+"(?P<user_agent>.*?)"$'

def extract_os_label(ua_string):
    ua = ua_string.lower()
    if 'windows' in ua: return 'Windows'
    if 'linux' in ua: return 'Linux'
    if 'ubuntu' in ua: return 'Ubuntu'
    if 'mac' in ua: return 'MacOS'
    return 'Unknown'

def process_log_to_numerical(file_path, label):
    raw_data = []
    with open(file_path, 'r') as f:
        for line in f:
            match = re.match(LOG_PATTERN, line.strip())
            if match:
                raw_data.append(match.groupdict())
    
    if not raw_data:
        print(f'ERROR: No lines matched regex in {file_path}')
        return pd.DataFrame()
    
    df = pd.DataFrame(raw_data)
    df['session_id'] = df['session_id'].replace('-', 'none')
    
    # Convert timestamp string to datetime object
    df['dt_obj'] = pd.to_datetime(df['timestamp'], format='%d/%b/%Y:%H:%M:%S %z')
    
    # Convert to Unix Timestamp (Seconds) - This is your "Date.now()" equivalent
    df['unix_timestamp'] = df['dt_obj'].astype('int64') // 10**9

    # 2. AGGREGATION: Grouping into behavioral sessions
    sessions = df.groupby('session_id').agg(
        total_hits=('dt_obj', 'count'),
        unique_paths=('path', 'nunique'),
        start_ts=('dt_obj', 'min'),
        end_ts=('dt_obj', 'max'),
        # New requested fields
        unix_now=('unix_timestamp', 'first'), # The "Date.now()" of the first hit
        method_raw=('method', 'first'),
        referer_raw=('referer', 'first'),
        ua_raw=('user_agent', 'first')
    ).reset_index()

    # 3. NUMERICAL FEATURE ENGINEERING
    # RPM Calculation
    duration_min = (sessions['end_ts'] - sessions['start_ts']).dt.total_seconds() / 60
    sessions['rpm'] = sessions['total_hits'] / duration_min.replace(0, 1)

    # 4. ENCODING STRINGS TO NUMBERS
    # OS Encoding
    os_map = {'Windows': 1, 'Linux': 2, 'Ubuntu': 3, 'MacOS': 4, 'Unknown': 0}
    sessions['os_encoded'] = sessions['ua_raw'].apply(extract_os_label).map(os_map)
    
    # Method Encoding (Label Encoding)
    # GET=0, POST=1, others=2
    method_map = {'GET': 0, 'POST': 1}
    sessions['method_encoded'] = sessions['method_raw'].map(lambda x: method_map.get(x, 2))
    
    # Referer Encoding (Binary: 1 if exists, 0 if direct "-")
    sessions['has_referer'] = sessions['referer_raw'].apply(lambda x: 0 if x == '-' else 1)

    # 5. FINAL SELECTION: Only Numerical Columns
    final_df = sessions[[
        'unix_now',       # Date.now() equivalent
        'method_encoded', # GET/POST/PUT as numbers
        'has_referer',    # Referer vs Direct as 1/0
        'rpm', 
        'unique_paths', 
        'os_encoded', 
        'total_hits'
    ]].copy()
    
    final_df['is_bot'] = label
    return final_df

# --- EXECUTION ---
import glob

# 1. Process all Human logs in the folder
human_files = glob.glob('web_bot_detection_dataset/phase1/data/web_logs/humans/*.log')
human_data = [process_log_to_numerical(f, 0) for f in human_files]
human_df = pd.concat(human_data, ignore_index=True)

# 2. Process all Bot logs in the folder
bot_files = glob.glob('web_bot_detection_dataset/phase1/data/web_logs/bots/*.log')
bot_data = [process_log_to_numerical(f, 1) for f in bot_files]
bot_df = pd.concat(bot_data, ignore_index=True)

# 3. Combine and Save
master_dataset = pd.concat([human_df, bot_df], ignore_index=True)
master_dataset.to_csv('final_numerical_dataset.csv', index=False)

print('SUCCESS: Full Numerical Dataset Created!')
print(f'Total rows: {len(master_dataset)}')
print(master_dataset.head())

SUCCESS: Full Numerical Dataset Created!
Total rows: 453
     unix_now  method_encoded  has_referer         rpm  unique_paths  \
0  1571924524               0            1   44.516129            11   
1  1571928988               0            1   79.200000            13   
2  1571928042               0            1  100.000000             8   
3  1571919772               0            1   60.000000             8   
4  1571928347               0            1   48.712871            21   

   os_encoded  total_hits  is_bot  
0           2          46       0  
1           2          33       0  
2           2          10       0  
3           2          13       0  
4           2         164       0  


In [53]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [54]:
df = pd.read_csv('final_numerical_dataset.csv')

In [55]:
X = df.drop('is_bot', axis = 1) #seperate features (X) and target (y)
y = df['is_bot']

In [56]:
#80/20 split for test and train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)


In [57]:
model = RandomForestClassifier(n_estimators = 100, random_state =42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [58]:
y_pred = model.predict(X_test)

In [59]:
print(f"--- Model Accuracy: {accuracy_score(y_test, y_pred) * 100:.2f}% ---")
print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nDetailed Report:")
print(classification_report(y_test, y_pred))

--- Model Accuracy: 100.00% ---

Confusion Matrix:
[[48  0]
 [ 0 43]]

Detailed Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        48
           1       1.00      1.00      1.00        43

    accuracy                           1.00        91
   macro avg       1.00      1.00      1.00        91
weighted avg       1.00      1.00      1.00        91



# Feature Engineering Guide: What You Can Extract

## Currently Captured (8 features):
1. `unix_now` - Timestamp of first request
2. `method_encoded` - HTTP method type
3. `has_referer` - Whether request came from referrer
4. `rpm` - Requests per minute
5. `unique_paths` - Number of different URLs accessed
6. `os_encoded` - Operating system type
7. `total_hits` - Total number of requests
8. `is_bot` - Label (0=human, 1=bot)

## NEW Features You Can Add (17 more):

### **A. User-Agent Intelligence (3 features)**
```
1. ua_is_headless (BINARY: 0 or 1)
   - Detects: "HeadlessChrome", "headless" in user agent
   - Bot signal: Strong (headless browsers = automated)
   - Source: User-Agent header
   
2. ua_length (NUMERIC)
   - Length of user agent string
   - Bot signal: Bots often have SHORT user agents (suspicious)
   - Humans: Longer, browser-specific strings
   - Source: User-Agent header
   
3. ua_has_mobile_keyword (BINARY: 0 or 1)
   - Detect: "Mobile", "Android", "iPhone", "iPad"
   - Bot signal: Mobile bots might not declare mobile
   - Source: User-Agent header
```

### **B. Referer Analysis (3 features)**
```
4. referer_is_search_engine (BINARY)
   - Domain check: google.com, bing.com, duckduckgo.com
   - Bot signal: LOW (legitimate source)
   - Source: Referer header
   
5. referer_is_social (BINARY)
   - Domain check: facebook.com, twitter.com, reddit.com
   - Bot signal: LOW (legitimate source)
   - Source: Referer header
   
6. referer_is_suspicious (BINARY)
   - Known bad domains, spam referrer lists
   - Bot signal: HIGH
   - Source: Referer header
```

### **C. Request Pattern Timing (4 features)**
```
7. request_interval_avg (NUMERIC: seconds)
   - Average time between requests
   - Formula: session_duration / (total_hits - 1)
   - Bot signal: Humans have variable intervals, bots are regular
   - Source: Timestamp between log entries
   
8. request_interval_variance (NUMERIC)
   - How much request timing varies
   - Bot signal: Low variance = bot (too regular)
   - Source: Timestamp between log entries
   
9. burst_count (NUMERIC)
   - Count of requests within 1 second
   - Bot signal: Multiple requests/second = bot
   - Source: Timestamp clustering
   
10. min_request_interval (NUMERIC: seconds)
    - Fastest request interval
    - Bot signal: Sub-second = likely bot
    - Source: Timestamp between log entries
```

### **D. Path Pattern Analysis (4 features)**
```
11. path_depth_avg (NUMERIC)
    - Average directory depth (e.g., /a = 1, /a/b/c = 3)
    - Bot signal: Crawlers access deep paths
    - Source: Path parsing
    
12. repeating_path_ratio (NUMERIC: 0.0-1.0)
    - % of requests to same path
    - Formula: (total_hits - unique_paths) / total_hits
    - Bot signal: Crawlers visit MANY different paths
    - Source: Path counting
    
13. parameter_count (NUMERIC)
    - Average URL parameters per request
    - Bot signal: Crawlers avoid complex parameters
    - Source: URL parsing
    
14. path_contains_admin (BINARY)
    - Access to /admin, /wp-admin, /api/admin
    - Bot signal: HIGH (suspicious reconnaissance)
    - Source: Path inspection
```

### **E. HTTP Status Patterns (3 features)**
```
15. error_4xx_ratio (NUMERIC: 0.0-1.0)
    - % of 400-499 status codes
    - Bot signal: HIGH (scanners hit 404s)
    - Source: Status code analysis
    
16. error_5xx_ratio (NUMERIC: 0.0-1.0)
    - % of 500-599 status codes
    - Bot signal: Can be HIGH or LOW
    - Source: Status code analysis
    
17. success_ratio (NUMERIC: 0.0-1.0)
    - % of 200-299 status codes
    - Bot signal: LOW success = bot/crawler
    - Source: Status code analysis
```

### **F. Bandwidth Analysis (2 features)**
```
18. avg_response_size (NUMERIC: bytes)
    - Average bytes per response
    - Bot signal: Bots request differently
    - Source: Response size header
    
19. total_bandwidth (NUMERIC: bytes)
    - Sum of all response sizes
    - Bot signal: Reflects request patterns
    - Source: Response size header
```

### **G. Temporal Behavior (4 features)**
```
20. time_of_day (NUMERIC: 0-23)
    - Hour when session started
    - Bot signal: Off-hours activity suspicious
    - Source: Timestamp
    
21. day_of_week (NUMERIC: 0-6, Mon=0)
    - Day of week
    - Bot signal: Bots work weekends/off-hours
    - Source: Timestamp
    
22. is_off_hours (BINARY)
    - 1 if before 9am or after 5pm
    - Bot signal: HIGH (bots don't respect business hours)
    - Source: Timestamp
    
23. session_duration_sec (NUMERIC: seconds)
    - How long the session lasted
    - Bot signal: Very short = bot, very long = human
    - Source: end_ts - start_ts
```

### **H. Advanced Behavioral (4 features)**
```
24. method_variety (NUMERIC)
    - Count of different HTTP methods used
    - Bot signal: Bots mostly use GET
    - Source: Method distribution
    
25. ratio_post_to_get (NUMERIC)
    - POST requests / GET requests
    - Bot signal: Very different patterns
    - Source: Method distribution
    
26. returns_set_cookie (BINARY)
    - Whether responses contain Set-Cookie header
    - Bot signal: Humans accept cookies, bots might ignore
    - Source: Response headers (from access logs)
    
27. cache_control_ignored (BINARY)
    - Requests repeated to same path quickly (respects Cache-Control)
    - Bot signal: Bots ignore caching
    - Source: Path timestamps
```

# Can You Move Forward? YES - Here's The Complete Pipeline

## The Full Architecture

```
┌─────────────────────────────────────────────────────────────────┐
│ PHASE 1: TRAINING (What you're doing now)                       │
├─────────────────────────────────────────────────────────────────┤
│                                                                   │
│  Access Logs (Historical)                                        │
│  ├─ humans/access_1.log  (known humans)                         │
│  └─ bots/access_advanced_bots.log  (known bots)                 │
│         ↓                                                         │
│  Feature Extraction (31 features)                                │
│         ↓                                                         │
│  master_dataset.csv (186 rows labeled 0 or 1)                   │
│         ↓                                                         │
│  Train/Test Split (70/30)                                        │
│         ↓                                                         │
│  Train ML Model (RandomForest, LogisticRegression, etc.)        │
│         ↓                                                         │
│  Evaluate: Accuracy, Precision, Recall, F1-Score                │
│         ↓                                                         │
│  ✅ Save trained model to file (.pkl, .joblib)                  │
│                                                                   │
└─────────────────────────────────────────────────────────────────┘

┌─────────────────────────────────────────────────────────────────┐
│ PHASE 2: REAL-TIME DETECTION (Production)                       │
├─────────────────────────────────────────────────────────────────┤
│                                                                   │
│  User makes request through proxy.js                             │
│         ↓                                                         │
│  eventStore.js records: path, method, user_agent, referer, etc. │
│         ↓                                                         │
│  Feature Extraction (calculate same 31 features):               │
│    - rpm (from past 60s of events)                              │
│    - unique_paths                                                │
│    - ua_is_headless                                              │
│    - time_of_day                                                 │
│    - etc...                                                      │
│         ↓                                                         │
│  Load trained model                                              │
│         ↓                                                         │
│  model.predict(user_features) → [0 or 1]                        │
│         ↓                                                         │
│  DECISION:                                                       │
│    - 0 = HUMAN → Allow traffic                                  │
│    - 1 = BOT → Block or Challenge                               │
│         ↓                                                         │
│  ✅ Update Redis cache & global reputation                      │
│                                                                   │
└─────────────────────────────────────────────────────────────────┘
```

## Key Questions & Answers

### **Q1: Can I actually use the dataset I created?**
**YES, 100%** ✅

Your dataset:
- ✅ Has labeled data (humans vs bots)
- ✅ Has behavioral features that matter (RPM, unique paths, user agent patterns)
- ✅ Matches what your proxy system actually tracks
- ✅ 186 rows is small but workable for initial model

### **Q2: Will my proxy features match the training features?**
**YES, with caveats** ⚠️

**What matches perfectly:**
- rpm, unique_paths, total_hits ✅
- os_encoded, method_encoded ✅
- has_referer, time_of_day ✅
- ua_is_headless ✅

**What you need to calculate in proxy:**
- These features need to be calculated from RECENT events (last 60s, not full session)
- Example: For RPM, calculate from `getEvents(tenant.id, ip)` in past 60s

### **Q3: How do I integrate the model into proxy.js?**
**Three-step process:**

1. **Train model in Python** (notebook)
   - Save as: `ml_model.joblib`
   
2. **Use model in Node.js** (proxy.js)
   - Option A: Call Python subprocess
   - Option B: Use ONNX to convert to JavaScript
   - Option C: Create a separate ML microservice
   
3. **Calculate features in real-time**
   - Same logic as training, but on fresh events

### **Q4: What accuracy should I expect?**
**Depends on data quality:**

With your current dataset (186 rows):
- Expected accuracy: 65-85%
- Why: Small dataset, but clear signal

With more data (1000+ rows):
- Expected accuracy: 85-95%
- You have multiple log files - use them all!

### **Q5: Is 31 features too many?**
**NO, but you can start with fewer** 📊

Start with these 10 "core" features:
1. rpm
2. unique_paths
3. total_hits
4. ua_is_headless
5. time_of_day
6. has_referer
7. error_4xx_ratio
8. session_duration_sec
9. os_encoded
10. method_encoded

Then add more if accuracy is low.